In [1]:
import owlready2 as owl
from pathlib import Path

onto = owl.get_ontology('/home/ivanstefanov/Repositories/bar-ontology/bar.owl')

### Top level concepts and roles

In [2]:
with onto:
    class Drink(owl.Thing): pass
    class Garnish(owl.Thing): pass
    class Ingredient(owl.Thing): pass
    class Technique(owl.Thing): pass
    
    class has_abv(onto.Drink >> int, owl.FunctionalProperty): pass 
    class has_for_ingredient(onto.Drink >> onto.Ingredient): pass

### Drink types with restrictions

In [3]:
with onto:
    class AlcoholicDrink(onto.Drink, onto.Ingredient): 
        equivalent_to = [onto.Drink & owl.Not(has_abv.value(0))]
        
    class NonAlcoholicDrink(onto.Drink, onto.Ingredient): 
        equivalent_to = [onto.Drink & has_abv.value(0)]
        
    class Cocktail(onto.Drink): 
        equivalent_to = [onto.Drink & has_for_ingredient.min(2, onto.Ingredient)]

### Other drink concepts

In [4]:
with onto:
    class Beer(AlcoholicDrink, onto.Ingredient): pass
    class Liqueur(AlcoholicDrink, onto.Ingredient): pass
    class Spirit(AlcoholicDrink, onto.Ingredient): pass
    class Wine(AlcoholicDrink, onto.Ingredient): pass
    owl.AllDisjoint([Beer, Liqueur, Spirit, Wine])
    
    class CarbonatedNonAlcoholicDrink(NonAlcoholicDrink): pass
    class StillNonAlcoholicDrink(NonAlcoholicDrink): pass
    owl.AllDisjoint([CarbonatedNonAlcoholicDrink, StillNonAlcoholicDrink])
    
    class Juice(StillNonAlcoholicDrink): pass
    
    class Redbull(CarbonatedNonAlcoholicDrink): pass
    class Soda(CarbonatedNonAlcoholicDrink): pass
    owl.AllDisjoint([Redbull, Soda])
    
    class Syrup(onto.Ingredient): pass
    class EggWhite(onto.Ingredient): pass
    owl.AllDisjoint([Syrup, EggWhite])

### Cocktail roles

In [5]:
with onto:
    class is_made_with_technique(Cocktail >> Technique): pass
    class has_for_garnish(Cocktail >> Garnish): pass
    class has_for_spirits(has_for_ingredient):
        domain = [Cocktail]
        range = [Spirit]
        
    class has_quantity(onto.Ingredient >> int, owl.FunctionalProperty): pass
    class has_making_process(Drink >> str): pass
    class has_story(has_making_process, owl.FunctionalProperty): 
        domain = [Cocktail]
        range = [str]
        
    class is_ingredient_of(onto.Ingredient >> Cocktail):
        inverse_property = has_for_ingredient
        
    class is_used_in_making_of(Technique >> Cocktail):
        inverse_property = is_made_with_technique
        
    class is_used_to_garnish(Garnish >> Cocktail):
        inverse_property = has_for_garnish

### Specific spirits

In [6]:
with onto:
    class Vodka(Spirit): 
        has_making_process = Path('stories/vodka.txt').read_text()
        has_abv = 37
    class Gin(Spirit): 
        has_making_process = Path('stories/gin.txt').read_text()
        has_abv = 40
        
    class Jagermeister(Liqueur): 
        has_making_process = Path('stories/jager.txt').read_text()
        has_abv = 50

### Cocktails by spirits

In [7]:
with onto:
    class VodkaCocktail(Cocktail):
        equivalent_to = [Cocktail & has_for_ingredient.some(Vodka)]
        
    class GinCocktail(Cocktail):
        equivalent_to = [Cocktail & has_for_ingredient.some(Gin)]
        
    class HardCocktail(Cocktail):
        equivalent_to = [Cocktail & has_for_ingredient.min(2, AlcoholicDrink)]

### Technique constants

In [8]:
building = Technique('building')
shaking = Technique('shaking')
dry_shaking = Technique('dry-shaking')
straining = Technique('straining')
stirring = Technique('stirring')
bombing = Technique('bombing')

owl.AllDifferent([building, shaking, dry_shaking, straining, stirring, bombing])

AllDisjoint([bar.building, bar.shaking, bar.dry-shaking, bar.straining, bar.stirring, bar.bombing])

### Cocktail constants

In [9]:
vodka_60ml = Vodka('60ml-of-vodka', has_quantity=60)
lime_juice_30ml = Juice('30ml-of-lime-juice', has_quantity=30)
simple_syrup_15ml = Syrup('15ml-of-simple-syrup', has_quantity=15)
lime_wheel = Garnish('lime-wheel')
gimlet = VodkaCocktail('gimlet')
gimlet.has_for_ingredient = [vodka_60ml, lime_juice_30ml, simple_syrup_15ml]
gimlet.is_made_with_technique = [shaking, straining]
gimlet.has_for_garnish = [lime_wheel]
gimlet.has_story = Path('stories/vodka-gimlet.txt').read_text()

In [10]:
gin_60ml = Gin('30ml-of-gin', has_quantity=60)
lemon_juice_30ml = Juice('30ml-of-lemon-juice', has_quantity=30)
egg_white = EggWhite('about-1-ounce-of-egg-white', has_quantity=1)
soda = Soda('soda')
gin_fizz = GinCocktail('gin-fizz')
gin_fizz.has_for_ingredient = [gin_60ml, lemon_juice_30ml, egg_white, soda]
gin_fizz.is_made_with_technique = [building, stirring]
gin_fizz.has_story = Path('stories/gin-fizz.txt').read_text()

In [11]:
ginger_beer_90ml = CarbonatedNonAlcoholicDrink('90ml-of-ginger-beer', has_quantity=90)
moscow_mule = VodkaCocktail('moscow-mule')
moscow_mule.has_for_ingredient = [vodka_60ml, lime_juice_30ml, ginger_beer_90ml]
moscow_mule.is_made_with_technique = [building, stirring]
moscow_mule.has_for_garnish = [lime_wheel]
moscow_mule.has_story = Path('stories/moscow-mule.txt').read_text()

In [12]:
jager_45ml = Jagermeister('45ml-of-jagermeister', has_quantity=45)
redbull_90ml = Redbull('about-half-a-can-of-redbull', has_quantity=90)
jager_bomb = Cocktail('jager-bomb')
jager_bomb.has_for_ingredient = [jager_45ml, redbull_90ml]
jager_bomb.is_made_with_technique = [bombing]

### Reasoning examples

In [13]:
unknown_cocktail1 = Cocktail('unknown-cocktail1')
unknown_cocktail1.has_for_ingredient = [vodka_60ml, soda]
unknown_cocktail1.is_made_with_technique = [building]

In [14]:
unknown_cocktail2 = Cocktail('unknown_cocktail2')
unknown_cocktail2.has_for_ingredient = [Beer, Jagermeister]
unknown_cocktail2.is_made_with_technique = [bombing]

In [15]:
unknown_cocktail3 = Cocktail('unknown_cocktail3')
unknown_cocktail3.has_for_ingredient = [vodka_60ml, gin_60ml, redbull_90ml, soda]

In [39]:
with onto:
    owl.sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /home/ivanstefanov/.pyenv/versions/3.10.1/envs/ml/lib/python3.10/site-packages/owlready2/pellet/httpcore-4.2.2.jar:/home/ivanstefanov/.pyenv/versions/3.10.1/envs/ml/lib/python3.10/site-packages/owlready2/pellet/antlr-3.2.jar:/home/ivanstefanov/.pyenv/versions/3.10.1/envs/ml/lib/python3.10/site-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/home/ivanstefanov/.pyenv/versions/3.10.1/envs/ml/lib/python3.10/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/ivanstefanov/.pyenv/versions/3.10.1/envs/ml/lib/python3.10/site-packages/owlready2/pellet/log4j-api-2.19.0.jar:/home/ivanstefanov/.pyenv/versions/3.10.1/envs/ml/lib/python3.10/site-packages/owlready2/pellet/antlr-runtime-3.2.jar:/home/ivanstefanov/.pyenv/versions/3.10.1/envs/ml/lib/python3.10/site-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/home/ivanstefanov/.pyenv/versions/3.10.1/envs/ml/lib/python3.10/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/ivanstefanov

* Owlready * Adding relation bar.60ml-of-vodka has_making_process Traditional vodka is made from two raw materials: water and ethanol \\nfrom the fermentation of cereal grains (like wheat, sorghum, or rye).\\nMany vodka brands use other base ingredients (like potatoes and \\nsugar beets) and additives (like botanicals and spices) to achieve \\ndistinctive character in their liquor. After fermenting and \\ndistilling, vodka undergoes a filtration and refining process to \\nremove impurities and achieve a smooth mouthfeel.
* Owlready * Adding relation bar.45ml-of-jagermeister has_making_process Jägermeister\\\'s ingredients include 56 herbs, fruits, roots, and \\nspices, including citrus peel, licorice, anise, poppy seeds, \\nsaffron, ginger, juniper berries, and ginseng.[14] These ingredients\\nare ground, then steeped in water and alcohol for two to three days.\\nThis mixture is filtered and stored in oak barrels for about a year.\\nThen the liqueur is filtered again, and mixed with su

* Owlready2 * Pellet took 1.575838565826416 seconds
* Owlready * Reparenting bar.Beer: {bar.Ingredient, bar.AlcoholicDrink} => {bar.AlcoholicDrink}
* Owlready * Reparenting bar.Liqueur: {bar.Ingredient, bar.AlcoholicDrink} => {bar.AlcoholicDrink}
* Owlready * Reparenting bar.Spirit: {bar.Ingredient, bar.AlcoholicDrink} => {bar.AlcoholicDrink}
* Owlready * Reparenting bar.Wine: {bar.Ingredient, bar.AlcoholicDrink} => {bar.AlcoholicDrink}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [17]:
onto.get_parents_of(unknown_cocktail1)

[bar.Cocktail, bar.VodkaCocktail]

In [18]:
onto.get_parents_of(unknown_cocktail2)

[bar.Cocktail]

In [19]:
onto.get_parents_of(unknown_cocktail3)

[bar.Cocktail, bar.GinCocktail, bar.HardCocktail, bar.VodkaCocktail]